# 用 tf.data 加载 CSV 数据
这篇教程通过一个示例展示了怎样将 CSV 格式的数据加载进 tf.data.Dataset。

这篇教程使用的是泰坦尼克号乘客的数据。模型会根据乘客的年龄、性别、票务舱和是否独自旅行等特征来预测乘客生还的可能性。

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

D:\Anaconda\envs\tensorflow2.0\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\envs\tensorflow2.0\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\envs\tensorflow2.0\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\envs\tensorflow2.0\lib\site-packages\tensorflow\python\framework\dtypes.py:5

In [16]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [17]:
# 让 numpy 数据更易读。
np.set_printoptions(precision=3, suppress=True)

In [18]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']
def get_dataset(file_path):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=12, # 为了示例更容易展示，手动设置较小的值
#       label_name=CSV_COLUMNS,
      na_value="?",
      num_epochs=1,
      ignore_errors=True)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

# tf.data load numpy

In [20]:
DATA_URL = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'

path = tf.keras.utils.get_file('mnist.npz', DATA_URL)
with np.load(path) as data:
  train_examples = data['x_train']
  train_labels = data['y_train']
  test_examples = data['x_test']
  test_labels = data['y_test']
    
train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))

BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
                
model.fit(train_dataset, epochs=10)

Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
938/938 [==============================] - 4s 5ms/step - loss: 9.9235 - sparse_categorical_accuracy: 0.3818
Epoch 2/10
938/938 [==============================] - 3s 3ms/step - loss: 8.7306 - sparse_categorical_accuracy: 0.4569
Epoch 3/10
938/938 [==============================] - 3s 3ms/step - loss: 8.5687 - sparse_categorical_accuracy: 0.4672
Epoch 4/10
938/938 [==============================] - 3s 3ms/step - loss: 8.5179 - sparse_categorical_accuracy: 0.4707
Epoch 5/10
938/938 [==============================] - 3s 3ms/step - loss: 8.4868 - sparse_categorical_accuracy: 0.4728
Epoch 6/10
938/938 [==============================] - 3s 3ms/step - loss: 8.4430 - sparse_categorical_accuracy: 0.4755
Epoch 7/10
938/938 [==============================] - 3s 4ms/step - loss: 8.3749 - sparse_categorical_accuracy: 0.4798
Epoch 8/10
938/938 [==============================] - 3s 4ms/step - loss:

# 使用 tf.data 加载 pandas dataframes

In [23]:
import pandas as pd

csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/applied-dl/heart.csv')
df = pd.read_csv(csv_file)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [24]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal         object
target        int64
dtype: object

In [25]:
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df.thal.cat.codes

In [26]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,2,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,4,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0


In [27]:
target = df.pop('target')
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))
dataset

<TensorSliceDataset shapes: ((13,), ()), types: (tf.float64, tf.int64)>

In [30]:
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   2. ], Target: 0
Features: [ 67.    1.    4.  160.  286.    0.    2.  108.    1.    1.5   2.    3.
   3. ], Target: 1
Features: [ 67.    1.    4.  120.  229.    0.    2.  129.    1.    2.6   2.    2.
   4. ], Target: 0
Features: [ 37.    1.    3.  130.  250.    0.    0.  187.    0.    3.5   3.    0.
   3. ], Target: 0
Features: [ 41.    0.    2.  130.  204.    0.    2.  172.    0.    1.4   1.    0.
   3. ], Target: 0


In [31]:
train_dataset = dataset.shuffle(len(df)).batch(1)
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

model = get_compiled_model()
model.fit(train_dataset, epochs=15)

Epoch 1/15
303/303 [==============================] - 2s 5ms/step - loss: 4.2253 - accuracy: 0.7353
Epoch 2/15
303/303 [==============================] - 1s 3ms/step - loss: 4.2253 - accuracy: 0.7353
Epoch 3/15
303/303 [==============================] - 1s 3ms/step - loss: 4.2253 - accuracy: 0.7353
Epoch 4/15
303/303 [==============================] - 1s 3ms/step - loss: 4.2253 - accuracy: 0.7353
Epoch 5/15
303/303 [==============================] - 1s 3ms/step - loss: 4.2253 - accuracy: 0.7353
Epoch 6/15
303/303 [==============================] - 1s 3ms/step - loss: 4.2253 - accuracy: 0.7353
Epoch 7/15
303/303 [==============================] - 1s 3ms/step - loss: 4.2253 - accuracy: 0.7353
Epoch 8/15
303/303 [==============================] - 1s 3ms/step - loss: 4.2253 - accuracy: 0.7353
Epoch 9/15
303/303 [==============================] - 1s 3ms/step - loss: 4.2253 - accuracy: 0.7353
Epoch 10/15
303/303 [==============================] - 1s 3ms/step - loss: 4.2253 - accuracy: 0.7353

# 用 tf.data 加载图片

# 使用 tf.data 加载文本数据

In [32]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import tensorflow_datasets as tfds
import os

In [33]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)
  
parent_dir = os.path.dirname(text_dir)

parent_dir

811008/807992 [==============================] - 0s 1us/step


'C:\\Users\\liu\\.keras\\datasets'

In [34]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)  

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

In [38]:
labeled_data_sets[0]

<MapDataset shapes: ((), ()), types: (tf.string, tf.int64)>

In [35]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
  
all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

In [36]:
for ex in all_labeled_data.take(5):
  print(ex)

(<tf.Tensor: id=155488, shape=(), dtype=string, numpy=b'will go crying back to his widowed mother--he, Astyanax, who erewhile'>, <tf.Tensor: id=155489, shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: id=155492, shape=(), dtype=string, numpy=b'This spurious, genuine that. The bastard drove,'>, <tf.Tensor: id=155493, shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: id=155496, shape=(), dtype=string, numpy=b'Iphinous son of Dexius on the shoulder, as he was springing on to his'>, <tf.Tensor: id=155497, shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: id=155500, shape=(), dtype=string, numpy=b'When she heard this Juno smiled, and still smiling she laid the girdle'>, <tf.Tensor: id=155501, shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: id=155504, shape=(), dtype=string, numpy=b'they had come to Troy armed with bows, and with slings of twisted wool'>, <tf.Tensor: id=155505, shape=(), dtype=int64, numpy=2>)


## 文本编码成数字

In [39]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

17178

In [40]:
# 样本编码
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)
encoded_example = encoder.encode(example_text)
print(encoded_example)

b'will go crying back to his widowed mother--he, Astyanax, who erewhile'
[13178, 11315, 8703, 1508, 14427, 5122, 2753, 4107, 5591, 3631, 813, 7823]


In [44]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

def encode_map_fn(text, label):
  # py_func doesn't set the shape of the returned tensors.
  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))

  # `tf.data.Datasets` work best if all components have a shape set
  #  so set the shapes manually: 
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label


all_encoded_data = all_labeled_data.map(encode_map_fn)

In [45]:
# 分离数据
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE)

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

TypeError: padded_batch() missing 1 required positional argument: 'padded_shapes'

In [ ]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]

vocab_size += 1